<a href="https://colab.research.google.com/github/cam2149/MachineLearningV/blob/main/taller/EDAv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Equipo**

- Nicolás Colmenares

- Carlos Martinez

1. Análisis Exploratorio de Datos (EDA) - 0.5 pts

  -  Carga y limpieza de los datos.

  - Visualización de tendencias y patrones de dengue.

  -  Análisis de correlaciones entre variables.

**Situación:**
Una ciudad enfrenta un aumento significativo de casos de dengue, con una tasa de incidencia que supera el promedio nacional.
La anticipación de brotes es crucial para implementar medidas preventivas y reducir la propagación de la enfermedad.

**Objetivo:**
Desarrollar un modelo predictivo utilizando redes neuronales para pronosticar futuros brotes de dengue en cada barrio de la ciudad.
Utilizar una base de datos histórica de casos de dengue desde 2015 hasta 2022 para entrenar el modelo.
Anticiparse a los brotes con al menos 3 semanas de anticipación.

**Finalidad:**
Permitir a las autoridades de salud pública tomar acciones oportunas, como:
Preparar a las instituciones prestadoras de salud (IPS).
Gestionar recursos (carros fumigadores, limpieza de sumideros).
Capacitar a la comunidad.

*   Análisis exploratorio de datos de la serie temporal
*   Explicar el comportamiento de la serie temporal en términos cualitativos y cuantitativos para desarrollar intuición para la selección del modelo
*    Identificar los modelos candidatos y los posibles parámetros del modelo que se pueden utilizar basándose en los hallazgos del análisis exploratorio de datos

# 0. Configuraciones de Colab

Mover Kaggle.json a la ubicación correcta después de subirlo

In [ ]:
#Estas líneas son comandos de shell que se ejecutan dentro del Jupyter notebook. Se usan para configurar las credenciales de la API de Kaggle, que son necesarias para descargar conjuntos de datos (datasets) desde Kaggle.

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!rm -rf /content/kaggle/output
!rm -rf /content/kaggle/input

Descargar dataset de la competencia

In [ ]:
!kaggle competitions download -c aa-v-2025-i-pronosticos-nn-rnn-cnn

In [ ]:
!mkdir -p /content/kaggle/output
!mkdir -p /content/kaggle/input

In [ ]:
!mv aa-v-2025-i-pronosticos-nn-rnn-cnn.zip /content/kaggle/input

In [ ]:
!unzip /content/kaggle/input/aa-v-2025-i-pronosticos-nn-rnn-cnn.zip -d /content/kaggle/input/

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#/kaggle/input
import os
for dirname, _, filenames in os.walk('/content/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# 1. Imports

In [ ]:
!pip install altair

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import altair as alt
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns
import statsmodels.api as sm
import scipy
from scipy.stats import anderson
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
#Printing library versions
print('Pandas:', pd.__version__)
print('Statsmodels:', sm.__version__)
print('Scipy:', scipy.__version__)
print('Matplotlib:', mpl.__version__)
print('Seaborn:', sns.__version__)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
np.random.seed(786)

# 2. Configs

In [ ]:
config = {
    "TRAIN_DIR": '/content/kaggle/input/df_train.parquet',
    "TEST_DIR": '/content/kaggle/input/df_test.parquet',
    "SUBMISSION_DIR": '/content/sample_submission.csv'
}

# Exploración

## Diccionario

train.parquet - El conjunto de datos de entrenamiento
test.parquet - El conjunto de datos de prueba
sample_submission.csv - un ejemplo de un archivo a someter en la competencia

| **Variable**         | **Descripción**                                                                                      |
|-----------------------|------------------------------------------------------------------------------------------------------|
| id_bar               | identificador único del barrio                                                                      |
| anio                 | Año de ocurrencia                                                                                   |
| semana               | Semana de ocurrencia                                                                               |
| Estrato              | Estrato socioeconómico del barrio                                                                   |
| area_barrio          | Área del barrio en km²                                                                             |
| dengue               | Conteo de casos de dengue                                                                          |
| concentraciones      | Cantidad de visitas e intervención a lugares de concentración humana (Instituciones)                |
| vivienda             | Conteo de las visitas a viviendas a revisión y control de criaderos                                 |
| equipesado           | Conteo de las fumigaciones con Maquinaria Pesada                                                   |
| sumideros            | Conteo de las intervenciones a los sumideros                                                       |
| maquina              | Conteo de las fumigaciones con motomochila                                                         |
| lluvia_mean          | Lluvia promedio en la semana i                                                                     |
| lluvia_var           | Varianza de la lluvia en la semana i                                                               |
| lluvia_max           | Lluvia máxima en la semana i                                                                       |
| lluvia_min           | Lluvia mínima en la semana i                                                                       |
| temperatura_mean     | Temperatura promedio en la semana i                                                                |
| temperatura_var      | Varianza de la temperatura en la semana i                                                          |
| temperatura_max      | Temperatura máxima en la semana i                                                                  |
| temperatura_min      | Temperatura mínima en la semana i                                                                  |


## Lectura del dataset de entrenamiento

In [ ]:
# Assuming 'series_test.parquet' is in the current directory or you provide the correct path
try:
    train_df = pd.read_parquet(config["TRAIN_DIR"]).iloc[:,1:]
    print(train_df.describe())
    print(train_df.info())
    print(train_df.shape)
except FileNotFoundError:
    print("Error: 'series_train.parquet' not found. Please make sure the file exists in the current directory or provide the correct path.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
train_df.head()

In [ ]:
from datetime import datetime, timedelta

# Define la función para obtener el último día de la semana
def get_last_day_of_week(year, week):
    first_day_of_year = datetime(year, 1, 1)
    first_day_of_week = first_day_of_year + timedelta(days=(week - 1) * 7 - first_day_of_year.weekday())
    last_day_of_week = first_day_of_week + timedelta(days=6)
    return last_day_of_week

# Aplicar la función al dataset
train_df['ultimo_dia_semana'] = train_df.apply(lambda row: get_last_day_of_week(int(row['anio']), int(row['semana'])), axis=1)

# Imprimir el DataFrame actualizado
print(train_df)

In [ ]:
import pandas as pd
# Convert 'ultimo_dia_semana' to datetime if it's not already
train_df['ultimo_dia_semana'] = pd.to_datetime(train_df['ultimo_dia_semana'])
# Set 'ultimo_dia_semana' as the index
train_df = train_df.set_index('ultimo_dia_semana')


Todas las columnas son númericas

In [ ]:
train_df.describe().T

In [ ]:
missing_values = train_df.isnull().sum()
missing_values[missing_values > 0]

In [ ]:
#Any missing data?
print("missing_data:", train_df.isna().sum())
print("unique dates:", train_df.index.nunique())

In [ ]:
#Contando el número de valores para cada trimestre y año. Las columnas n son trimestres.
#Aquí, cada trimestre y año tiene un valor, por lo que no hay duplicados.
pd.crosstab(index=train_df.index.year, columns=train_df.index.quarter)

In [ ]:
train_df.index

In [ ]:
# Filter the DataFrame for cases where 'dengue' is greater than 25
filtered_df = train_df[train_df['dengue'] > 25]

# Create the text-based listing
output_text = ""
for index, row in filtered_df.iterrows():
    output_text += f"id_bar: {row['id_bar']}, Año: {row['anio']}, Casos de Dengue: {row['dengue']}\n"

# Print the output or save to a file
output_text
# Or, to save to a file:
# with open("dengue_cases_report.txt", "w") as f:
#     f.write(output_text)


***Exploración de Datos***

*   Una vez realizada la lectura del dataset de entrenamiento se revisan las estadísticas descriptivas (media, desviación estándar, mínimos, máximos, cuartiles) de los datos numéricos.
*   Se revisan el resumen de los tipos de datos y valores nulos en cada columna para obtener información de tipos de datos y valores perdidos, y las dimensiones del DataFrame.
*   Se procede a la creación de la columna **'ultimo_dia_semana'**: al DataFrame que representa el último día de la semana para cada fila, basándonos en el año y la semana proporcionados. Para poder indexar cronológicamente los datos, esta información se usa más adelante para establecer un índice de tiempo.
*   Número de filas y columnas del DataFrame `train_df`: [3680 rows x 20 columns]
*  Número de valores nulos por columna: [0]
*  Número de fechas únicas: [365]
*  Se crea una tabla cruzada que muestra el conteo de valores por año y trimestre Resultados del Conteo de valores por año y trimestre buscando desequilibrios o patrones recurrentes: En los años del 2015 al 2021 son mas homogeneos en la cantidad de datos recolectados del 2022 solo existen 10
* La variable Categorica Estrato: Contiene valores del 1 al 3
* Se hace una revisión los hitos mas significativos de los casos reportado de dengue teniendo como resultado.
   * id_bar: 2, Año: 2015, Casos de Dengue: 26.0
   * id_bar: 1, Año: 2015, Casos de Dengue: 27.0
   * id_bar: 1, Año: 2016, Casos de Dengue: 28.0
   * id_bar: 0, Año: 2020, Casos de Dengue: 29.0



**Nota**: Es fácil confundir la aleatoriedad con la estacionalidad. En el gráfico de recorrido aleatorio a continuación, puede parecer que los datos presentan cierta estacionalidad, pero no es así.

In [ ]:
np.random.seed(578)
steps = np.random.normal(0,1,500)

noise = pd.DataFrame({"x":steps})
wnoise_chart = alt.Chart(noise.reset_index()).mark_line().encode(
    x='index',
    y='x').properties(
    title="White Noise")

#Create random walk with N(0,1.5), 500 points

steps[0]=0
rwalk = pd.DataFrame({"x":100 + np.cumsum(steps)}).reset_index()
rwalk_chart = alt.Chart(rwalk).mark_line().encode(
    x='index',
    y=alt.Y('x', scale=alt.Scale(domain=(80,150)))).properties(
    title="Random Walk")


wnoise_chart | rwalk_chart

**Creación de un gráfico de barras para visualizar los casos de dengue**


In [ ]:
#Este segmento de código toma los datos de casos de dengue, genera un gráfico de barras que representa visualmente las fluctuaciones en los casos de dengue. Enfatiza la importancia de comprender el patrón intermitente de los brotes de la enfermedad.

demand = train_df['dengue']
range_values = np.arange(len(demand))
demanddf = pd.DataFrame({'y': demand, 'x': range_values})
alt.Chart(demanddf).mark_bar().encode(
    x='x',
    y='y').properties(
    title="Fluctuaciones de Casos del Dengue", width = 700)

In [ ]:
# Create a new 'anio_semana' column
train_df['anio_semana'] = train_df['anio'].astype(str) + '-' + train_df['semana'].astype(str).str.zfill(2)
train_df['anio_semana'] = pd.to_datetime(train_df['anio_semana'] + '-1', format='%Y-%W-%w')

# Get unique 'id_bar' values and sort them
unique_id_bars = sorted(train_df['id_bar'].unique())

# Calculate number of rows needed for 1 column (which is the same as the number of unique id_bars)
num_rows = len(unique_id_bars)

# Create a figure and a grid of subplots with 1 column
fig, axes = plt.subplots(num_rows, 1, figsize=(10, 6 * num_rows))

# Iterate and plot on each subplot
for i, id_bar in enumerate(unique_id_bars):
    id_bar_data = train_df[train_df['id_bar'] == id_bar]
    axes[i].plot(id_bar_data['anio_semana'], id_bar_data['dengue'])  # Use axes[i] for single column
    axes[i].set_xlabel('Año-Semana')
    axes[i].set_ylabel('Casos de Dengue')
    axes[i].set_title(f'id_bar: {id_bar}')

# Adjust spacing between subplots
plt.tight_layout()
plt.show()

## Visualizar la serie temporal

In [ ]:
# Create a new 'anio_semana' column
train_df['anio_semana'] = train_df['anio'].astype(str) + '-' + train_df['semana'].astype(str).str.zfill(2)
train_df['anio_semana'] = pd.to_datetime(train_df['anio_semana'] + '-1', format='%Y-%W-%w')

# Get unique 'id_bar' values and assign colors
unique_id_bars = train_df['id_bar'].unique()
colors = plt.cm.get_cmap('viridis', len(unique_id_bars))  # Use a colormap

# Create a single plot
fig, ax = plt.subplots(figsize=(10, 5))

# Plot data for each 'id_bar' with different colors
for i, id_bar in enumerate(unique_id_bars):
    id_bar_data = train_df[train_df['id_bar'] == id_bar]
    ax.plot(id_bar_data['anio_semana'], id_bar_data['dengue'],
            color=colors(i), label=f'id_bar: {id_bar}')

# Customize plot
ax.set_xlabel('Año-Semana')
ax.set_ylabel('Casos de Dengue')
ax.set_title('Tendencia de Casos de Dengue por id_bar (Color)')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Place legend outside

plt.tight_layout()
plt.show()

In [ ]:
plt.bar(train_df['ESTRATO'], train_df['dengue'])
plt.xlabel('Estrato')
plt.ylabel('Casos de Dengue')
plt.title('Casos de Dengue por Estrato')
plt.show()

## Matriz de correlación

In [ ]:
corr_matrix = train_df.iloc[:,:-1].corr()

# Define colors and positions for the gradient
colors = [(1, 0, 0), (0.6, 0.8, 1), (1, 0.5, 0)]  # Red, Light Blue, Orange in RGB
positions = [0, 0.6, 1]  # Positions for color transitions (0 to 1)

# Create custom colormap
cmap = LinearSegmentedColormap.from_list('custom_cmap', list(zip(positions, colors)))

# Create heatmap with custom colormap
sns.heatmap(corr_matrix,
            yticklabels=corr_matrix.columns,
            xticklabels=False,
            annot=False,
            cmap=cmap,
            vmin=-1, vmax=1,  # Set color range limits
            center=0)  # Center color at 0


plt.gca().spines['bottom'].set_visible(False)  # Hide the x-axis line

plt.title('Matriz de Correlación')
plt.show()

In [ ]:
# Filter correlations greater than 0.5 (absolute value)
high_correlations = corr_matrix[abs(corr_matrix) > 0.5]

# Print the filtered correlations in text format
print("Correlaciones mayores a 0.5:")
for col in high_correlations.columns:
    for index in high_correlations.index:
      if abs(high_correlations.loc[index,col]) > 0.5 and index != col:
        print(f"{index} - {col}: {high_correlations.loc[index,col]:.2f}")



Se presenta correlación entre **id_bar** y **area_barrio**, además de entre las variables de lluvia al igual que las de temperatura.

La mayor correlacción con **dengue** es **anio** y **semana**

In [ ]:
# Using statmodels: Subtracting the Trend Component
from statsmodels.tsa.seasonal import seasonal_decompose

# Check for zero and negative values in 'dengue' column
if (train_df['dengue'] <= 0).any():
    # If zero or negative values exist, use 'additive' model
    result_mul = seasonal_decompose(train_df['dengue'], model='additive', period=30)
else:
    # If no zero or negative values, use 'multiplicative' model
    result_mul = seasonal_decompose(train_df['dengue'], model='multiplicative', period=30)

detrended = train_df['dengue'].values - result_mul.trend
plt.plot(detrended)
plt.title('Dengue desestabilizados restando el componente de tendencia', fontsize=16)

La forma habitual de comprobar la estacionalidad de una serie temporal es graficarla y comprobar si existen patrones repetibles en intervalos de tiempo fijos. Por lo tanto, el tipo de estacionalidad se determina por el reloj o el calendario.
Sin embargo, si deseamos una evaluación más precisa de la estacionalidad, utilice el gráfico de la Función de Autocorrelación (ACF). Debido a la fuerte estacionalidad, el gráfico de la ACF suele revelar picos repetidos y definidos en los múltiplos de la ventana estacional.

In [ ]:
# Test for seasonality
from pandas.plotting import autocorrelation_plot

# Draw Plot
plt.rcParams.update({'figure.figsize':(10,6), 'figure.dpi':120})
autocorrelation_plot(train_df['dengue'].tolist())